<a href="https://colab.research.google.com/github/mandar-solanki/agents-course-ms/blob/main/unit2/code_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install smolagents -U
!pip install ddgs
!pip install 'smolagents[litellm]'


In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [2]:
from google.colab import userdata

In [3]:
llm_model_name = 'gemini/gemini-2.0-flash-lite'
api_key = userdata.get("GEMINI_API_KEY")

In [4]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel, LiteLLMModel

agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=LiteLLMModel(
    model_id=llm_model_name,
    api_key=api_key,
    num_ctx=1024,
))

agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search for the best music recommendations for a party at the Wayne's mansion.                                   │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.0-flash-lite ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = "music recommendations for a party at Wayne's mansion"                                                   
  results = web_search(query)                                                                                      
  print(results)                                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[HOUSE PARTY! - Good Times at Davey 
Wayne's](https://www.tripadvisor.com/ShowUserReviews-g32655-d7001888-r306375731-Good_Times_at_Davey_Wayne_s-Los_Ang
eles_California.html)
Good Times at Davey Wayne's : HOUSE PARTY ! - See 75 traveler reviews , 28 candid photos, and great deals for Los 
Angeles, CA, at Tripadvisor.

[Revisiting Wayne's World, A Rock'n'Roll Party That Still 
...](https://pitchfork.com/thepitch/revisiting-waynes-world-a-rocknroll-party-that-still-feels-familiar/)
May 7, 2020 — The 1992 adaptation of Myers' popular Saturday Night Live sketch extended his character's 
catchphrases and carefree, rock-star-worshipping ethos for 90 minutes.

[I want to expand my music taste. Please give album/artist 
...](https://www.reddit.com/r/MusicRecommendations/comments/1fyvz6l/i_want_to_expand_my_music_taste_please_give/)
God Smack . Audio Slave. Reo Speedwagon- HI Infidelity Disturbed- Down with the sickness, but try all their music. 
Chevelle. Mudvayne. The Eagles ...

[10 Best L.A. Bars and Restaurants With Live 
Music](https://goodtimesatdaveywaynes.com/10-best-l-a-bars-and-restaurants-with-live-music/)
This themed Houston Brothers bar, right down to the live music stage. Retro-style rockin' goes down here weekly , 
and the colorful drinks served compliment the ...

[Memorial Weekend Live Music at Old City Pub in Wildwood 
...](https://www.facebook.com/groups/619831895291217/posts/1710642062876856/)
Come on out & party with Animal House Band Saturday Afternoon Jam at Old City Pub 3301 Atlantic Ave. Wildwood, NJ 
Happy Memorial Weekend ALL AGES WELCOME

[What are some good tracks for house parties?](https://www.quora.com/What-are-some-good-tracks-for-house-parties)
1. Strobe - Deadmau5 · 2. Pjanoo - Eric Prydz · 3. Call on me - Eric Prydz · 4. I need your love - Calvin Harris · 
5. Levels - Avicii · 6. Fade into ...

[Good Times at Davey Wayne's (@gtdaveywaynes)](https://www.instagram.com/gtdaveywaynes/?hl=en)
Every Wednesday at 10PM, ABSOLUTELY FREE!! Bring Your Groovy Gals and Pals!! Dress Your Grooviest!! We're back to 
rock the house down!! #Electricrevival # ...

[Radar Love (From "Wayne's World 2")](https://open.spotify.com/track/5idnxUSWmoSD31arlQTnN7)
Recommended based on this song ; Radar Love (From "Pushing Tin ")Golden Raptor ; Driving 60s RockBobby Cole ; Entry
Of The Gladiators RockKen mukai ; Birds Are Bad ...

[unit2/smolagents/tool_calling_agents.ipynb · agents-course 
...](https://huggingface.co/agents-course/notebooks/blob/refs/pr/34/unit2/smolagents/tool_calling_agents.ipynb)
Final answer: For a party at Wayne's mansion, consider playing a mix of classic party hits and modern anthems to 
cater to various age groups. A recommended ...

[What are some songs you discovered through movies or 
...](https://www.reddit.com/r/MusicRecommendations/comments/1kch40n/what_are_some_songs_you_discovered_through_movi
es/)
" Beneath the ice" - Two steps from hell "Dirge" - Death in Vegas (I heard these two in crime shows). "Un ano de 
amor" and "Piensa en mi" ...

Out: None

[Step 1: Duration 1.52 seconds| Input tokens: 2,118 | Output tokens: 75]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("For a party at Wayne's mansion, a good music recommendation would be a mix of electronic (like     
  Daft Punk, Eric Prydz), pop (Miley Cyrus, Taylor Swift), and classic rock (Queen, The Rolling Stones). Include   
  some classic party anthems (e.g., 'This Is How We Do It', 'It Takes Two') to appeal to a wide audience.          
  Consider also including modern rock (The Strokes, Imagine Dragons) for the younger crowd.")                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: For a party at Wayne's mansion, a good music recommendation would be a mix of electronic (like Daft 
Punk, Eric Prydz), pop (Miley Cyrus, Taylor Swift), and classic rock (Queen, The Rolling Stones). Include some 
classic party anthems (e.g., 'This Is How We Do It', 'It Takes Two') to appeal to a wide audience. Consider also 
including modern rock (The Strokes, Imagine Dragons) for the younger crowd.

[Step 3: Duration 1.51 seconds| Input tokens: 11,225 | Output tokens: 299]

"For a party at Wayne's mansion, a good music recommendation would be a mix of electronic (like Daft Punk, Eric Prydz), pop (Miley Cyrus, Taylor Swift), and classic rock (Queen, The Rolling Stones). Include some classic party anthems (e.g., 'This Is How We Do It', 'It Takes Two') to appeal to a wide audience. Consider also including modern rock (The Strokes, Imagine Dragons) for the younger crowd."

In [5]:
from smolagents import CodeAgent, tool

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion (str): The type of occasion for the party. Allowed values are:
                        - "casual": Menu for casual party.
                        - "formal": Menu for formal party.
                        - "superhero": Menu for superhero party.
                        - "custom": Custom menu.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

agent = CodeAgent(tools=[suggest_menu], model=LiteLLMModel(
    model_id=llm_model_name,
    api_key=api_key,
))

agent.run("Prepare a formal menu for the party.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Prepare a formal menu for the party.                                                                            │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.0-flash-lite ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  menu = suggest_menu(occasion="formal")                                                                           
  print(menu)                                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
3-course dinner with wine and dessert.

Out: None

[Step 1: Duration 10.70 seconds| Input tokens: 2,135 | Output tokens: 48]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("3-course dinner with wine and dessert.")                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 3-course dinner with wine and dessert.

[Step 2: Duration 0.66 seconds| Input tokens: 4,397 | Output tokens: 96]

'3-course dinner with wine and dessert.'

In [6]:
from smolagents import CodeAgent, InferenceClientModel
import numpy as np
import time
import datetime

agent = CodeAgent(tools=[],
                  model=LiteLLMModel(model_id=llm_model_name,api_key=api_key,),
                  additional_authorized_imports=['datetime']
                  )

agent.run(
    """
    Alfred needs to prepare for the party. Here are the tasks:
    1. Prepare the drinks - 30 minutes
    2. Decorate the mansion - 60 minutes
    3. Set up the menu - 45 minutes
    4. Prepare the music and playlist - 45 minutes

    If we start right now, at what time will the party be ready?
    """
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Alfred needs to prepare for the party. Here are the tasks:                                                      │
│     1. Prepare the drinks - 30 minutes                                                                          │
│     2. Decorate the mansion - 60 minutes                                                                        │
│     3. Set up the menu - 45 minutes                                                                             │
│     4. Prepare the music and playlist - 45 minutes                                                              │
│                                                                                                                 │
│     If we start right now, at what time will the party be ready?                                                │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.0-flash-lite ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  drink_time = 30                                                                                                  
  decorate_time = 60                                                                                               
  menu_time = 45                                                                                                   
  music_time = 45                                                                                                  
                                                                                                                   
  total_minutes = drink_time + decorate_time + menu_time + music_time                                              
                                                                                                                   
  print(f"Total preparation time: {total_minutes} minutes")                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Total preparation time: 180 minutes

Out: None

[Step 1: Duration 1.14 seconds| Input tokens: 2,128 | Output tokens: 115]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The party will be ready in 3 hours from now.")                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: The party will be ready in 3 hours from now.

[Step 3: Duration 0.48 seconds| Input tokens: 7,120 | Output tokens: 249]

'The party will be ready in 3 hours from now.'

In [8]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel, VisitWebpageTool, FinalAnswerTool, Tool, tool

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion: The type of occasion for the party.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

@tool
def catering_service_tool(query: str) -> str:
    """
    This tool returns the highest-rated catering service in Gotham City.

    Args:
        query: A search term for finding catering services.
    """
    # Example list of catering services and their ratings
    services = {
        "Gotham Catering Co.": 4.9,
        "Wayne Manor Catering": 4.8,
        "Gotham City Events": 4.7,
    }

    # Find the highest rated catering service (simulating search query filtering)
    best_service = max(services, key=services.get)

    return best_service

class SuperheroPartyThemeTool(Tool):
    name = "superhero_party_theme_generator"
    description = """
    This tool suggests creative superhero-themed party ideas based on a category.
    It returns a unique party theme idea."""

    inputs = {
        "category": {
            "type": "string",
            "description": "The type of superhero party (e.g., 'classic heroes', 'villain masquerade', 'futuristic gotham').",
        }
    }

    output_type = "string"

    def forward(self, category: str):
        themes = {
            "classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
            "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
            "futuristic gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets."
        }

        return themes.get(category.lower(), "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic gotham'.")


# Alfred, the butler, preparing the menu for the party
agent = CodeAgent(
    tools=[
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        suggest_menu,
        catering_service_tool,
        SuperheroPartyThemeTool()
        ],
    model=LiteLLMModel(
        model_id=llm_model_name,
        api_key=api_key,
    ),
    max_steps=10,
    verbosity_level=2
)

agent.run("Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme        │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.0-flash-lite ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: I need to create a playlist for a villain masquerade party at Wayne's mansion. I will use the             
`superhero_party_theme_generator` tool to get some ideas for the music playlist and then use these ideas to find   
music.                                                                                                             
<code>                                                                                                             
party_theme = superhero_party_theme_generator(category="villain masquerade")                                       
print(party_theme)                                                                                                 
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  party_theme = superhero_party_theme_generator(category="villain masquerade")                                     
  print(party_theme)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.

Out: None

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: The previous attempt failed because of a parsing error. I will provide the final answer in a code block   
using the `final_answer` tool. I'll directly list the suggested playlist.                                          
<code>                                                                                                             
final_answer("Gotham Rogues' Ball Playlist:\n1. Classical/Orchestral themes\n2. Dark Electronic/Industrial\n3.     
Soundtracks (Batman movies)\n4. \"Villain\" themed music\n5. Masquerade Ball themed music")                        
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Gotham Rogues' Ball Playlist:\n1. Classical/Orchestral themes\n2. Dark Electronic/Industrial\n3.   
  Soundtracks (Batman movies)\n4. \"Villain\" themed music\n5. Masquerade Ball themed music")                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Gotham Rogues' Ball Playlist:
1. Classical/Orchestral themes
2. Dark Electronic/Industrial
3. Soundtracks (Batman movies)
4. "Villain" themed music
5. Masquerade Ball themed music

[Step 4: Duration 1.21 seconds| Input tokens: 13,963 | Output tokens: 575]

'Gotham Rogues\' Ball Playlist:\n1. Classical/Orchestral themes\n2. Dark Electronic/Industrial\n3. Soundtracks (Batman movies)\n4. "Villain" themed music\n5. Masquerade Ball themed music'